In [2]:
# %load first_cell.py
import os
home = os.environ['HOME']

import sys
sys.path = sys.path + [f'{home}/.conda/envs/condaenv/lib/python37.zip', 
                       f'{home}/.conda/envs/condaenv/lib/python3.7', 
                       f'{home}/.conda/envs/condaenv/lib/python3.7/lib-dynload', 
                       f'{home}/projects/waze_data_normalization/.env/lib/python3.7/site-packages']



%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml

from pathlib import Path


import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [3]:
from src import utils

In [4]:
conn = utils.connect_athena(path='../configs/athena.yaml')

In [21]:
try:
    config_path = os.environ['CONFIG_PATH']
except:
    config_path = '../configs/20191029-panama_plaza.yaml'
    
try:
    geometry_path = os.environ['GEOMETRY_PATH']
except:
    geometry_path = '../configs/20191114-medellin.json'

In [6]:
config = utils.get_config(config_path)
    
config['report_data_path'] = Path(f'/home/joaom/shared/spd-sdv-omitnik-waze/preprocessed/{config["slug"]}/{os.environ["MILLIS"]}/support_files/report_data')

In [10]:
utils.safe_create_path(config['report_data_path'])


In [7]:
roadtype_dict = {
    1: '1 - freeway',
    2: '2 - major highway',
    3: '3 - ramp',
    4: '4 - minor highway',
    5: '5 - primary street',
    6: '6 - street',
    7: '7 - others'
}

In [8]:
def query_and_save(query, name, config, conn):
    
    save_path = config['report_data_path'] / (name + '.csv')
    
    pd.read_sql_query(query, conn).to_csv(save_path, index=False)
    
    return save_path

In [9]:
#Adds support files folder

# Config File

In [11]:
print(open(config_path, 'r').read())

# Query

s3_path: s3://iadbprod-public-stata-as-a-service/spd-sdv-omitnik-waze/preprocessed
athena_database: spd_sdv_waze_preprocessing
slug: panama_plaza
raw_database: p-waze-parquet-waze
raw_table: jams


# Date Ranges

road_network: 2019-03-31
estimation_period_start: 2018-10-29
estimation_period_end: 2018-11-21
unbalanced_panel_start: 2018-10-29
unbalanced_panel_end: 2019-03-31

# Initial Filters

initial_filters:
    - speed >= 0
    # - blockingalertuuid is null # uncomment to filter blocked roads
    - length > 0
    
# Segments Matching Algorithm

buffer_size: 2.5 # in meters
maximum_angle_difference: 20 # in degrees

# Reference Point

# If provided, calculates, else None

# Paralelization 

partitioned_query: [4, 16]
polygon_resolution: 6 # H3
number_of_athena_jobs: 190

# Winsorization

lower_hard_bound: 10 # km/h
upper_hard_bound: 100 # km/h
lower_percentile: 0.01
upper_percentile: 0.99






# Polygon 

In [22]:
from keplergl import KeplerGl

polygon =  open(geometry_path, 'r').read()

    
map_1 = KeplerGl(data={"polygon": polygon})
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'polygon': '{\n   "type" : "FeatureCollection",\n   "features" : [{\n      "type":"Feature",\n …

# Segment Sample Size

In [33]:
name = 'roadtype_per_samplesize'

query = f"""
select sample_size, roadtype
from {config["athena_database"]}.{config["slug"]}_jams_ready_segments_complete
"""

save_path = query_and_save(query, name, config, conn)

In [34]:
df = pd.read_csv(save_path)
df['roadtype_name'] = df['roadtype'].apply(lambda x: roadtype_dict[x])

## Distribution of Segment Sample Size

In [35]:
df['sample_size'].iplot(
    kind='histogram',
    title='Distribution of Segment Sample Size',
    xTitle='Sample Size',
    yTitle='Frequency',
    bins=None,
    yaxis_type='log',)

## Cumulative Distribution of Segment Sample Size

In [36]:
df.groupby('sample_size').count().cumsum().divide(len(df) / 100)['roadtype'].iplot(
        title='Cumulative Distribution of Sample Size for 200k Segments (~20% ss)',
        xTitle='Sample Size',
        yTitle='Cummulative %',
        width=5,
        xaxis_type='log'
        )

## Distribution of Segment Sample Size per Roadtype

## Cumulative Distribution of Segment Sample Size per Roadtype

In [ ]:
df.groupby(['sample_size', 'roadtype_name']).count().reset_index()\
        .pivot(columns='roadtype_name', values='roadtype', index='sample_size')\
        .cumsum().divide(df.groupby('roadtype_name').count()['roadtype'] / 100).iplot( 
    title='Cumulative Distribution of Segment Sample Size per Roadtype',
    xTitle='Sample Size',
    yTitle='%',
    shape=(3,2),
    subplots=True,
    shared_xaxes=True,
    subplot_titles=True,
    legend=False,
    xaxis_type='log',
    mode='lines', 
    fill=True,
    width=0
    )

## Share of Roadtype per Segment Sample Size

In [ ]:
df['sample'] = pd.cut(df['sample_size'], bins=range(0, 150000, 1000)).apply(lambda x: x.right)
df1 = df.groupby(['sample', 'roadtype_name']).sum().reset_index().pivot(index='sample', columns='roadtype_name', values='sample_size').fillna(0)

df1.divide(df1.sum(1), axis='index').multiply(100).iplot( 
    kind='bar', barmode='stack',
    xTitle='Sample Size',
    yTitle='% by Road Type',
    title='Share of Roadtype per Segment Sample Size')

## Roadtype By level

In [ ]:
name = 'roadtype_by_level'

query = f"""
select roadtype, level, count(*) as counta
from {config["athena_database"]}.{config["slug"]}_jams_ready_estimations_open_final
group by roadtype, level
"""

save_path = query_and_save(query, name, config, conn)

In [ ]:
df = pd.read_csv(save_path)
df['roadtype_name'] = df['roadtype'].apply(lambda x: roadtype_dict[x])

In [ ]:
df.pivot(index='roadtype_name', columns='level', values='counta')\
.divide(df.pivot(index='roadtype_name', columns='level', values='counta').sum(1), 'index').multiply(100)\
.iplot(kind='bar', barmode='stack',
        title='Road Type x Level', xTitle='Roadtype', yTitle='Level Proportion (%)')

# Accuracy

## Accuracy vs. Sample Size of Weighted Freeflow Estimators

In [ ]:
name = 'estimations_accuracy'

query = f"""
select sample_size, 
       avg(avg_accuracy) as freeflow, 
       avg(roadtype_accuracy) as roadtype_freeflow,
       avg(greatest(avg_accuracy, roadtype_accuracy)) as best_freeflow
from (
    select *
    from {config["athena_database"]}.{config["slug"]}_jams_ready_estimations_accuracy)
group by sample_size
order by sample_size
"""

save_path = query_and_save(query, name, config, conn)

In [ ]:
df = pd.read_csv(save_path)

In [ ]:
df.set_index('sample_size').rolling(200).mean().iplot(    
    xTitle='Estimation Sample Size',
    yTitle='Accuracy of predicting level',
    title='Accuracy x Estimation Sample Size w/ Rolling Mean 200 window (Weighted)')

In [ ]:
df.set_index('sample_size').rolling(1000).mean().iplot(    
    xTitle='Estimation Sample Size',
    yTitle='Accuracy of predicting level',
    title='Accuracy x Estimation Sample Size w/ Rolling Mean 1000 window (Weighted)')

## Accuracy vs. Sample Size of Weighted Freeflow Estimators By Roadtypes

In [ ]:
name = 'estimations_accuracy_roadtype'

query = f"""
select sample_size, roadtype, 
avg_accuracy as freeflow, roadtype_accuracy as roadtype_freeflow, best_accuracy as best_freeflow
from {config["athena_database"]}.{config["slug"]}_jams_ready_estimations_accuracy_roadtype
order by sample_size
"""

save_path = query_and_save(query, name, config, conn)

In [ ]:
df = pd.read_csv(save_path)
# df['roadtype_name'] = df['roadtype'].apply(lambda x: roadtype_dict[x])

In [ ]:
rolling = list(map(int, df.query('sample_size < 100000').groupby('roadtype').count()['sample_size'].divide(10)))
groups = list(map(str, np.sort(df['roadtype'].unique())))

# Adds figures
figs = [df.query(f"roadtype == '{group}'")\
        .query('sample_size < 100000')\
        .set_index('sample_size')\
        .drop(['roadtype'], 1)\
        .rolling(rolling[i]).mean()\
        .figure()
     for i, group in enumerate(groups)]

# Add legend group to same traces 
for i in range(len(groups)):
    for j in range(len(figs[i]['data'])):
        figs[i]['data'][j]['legendgroup'] = str(j)
        figs[i]['data'][j]['showlegend'] = i == 0

# Instantiate figure as subplot
figure = cf.subplots(figs, shape=(len(groups), 1), 
                     subplot_titles=[ 'Roadtype ' + roadtype_dict[int(group)] + f' (Rolling Window {rolling[i]})' 
                                     for i, group in enumerate(groups)], 
                     vertical_spacing=0.05)

# Adjust height and add overall title
figure['layout']['height'] = len(groups) * 300
figure['layout']['title'] = 'Accuracy x Sample Size by Level (Weighted)'


# Edits layout of each subplot
for i in range(1, len(groups) + 1):
    
    if i == 1:
        i = ''
        
    figure['layout'][f'yaxis{i}']['title'] = 'Accuracy'
    figure['layout'][f'yaxis{i}']['range'] = [0.2, 1]

In [ ]:
plotly.offline.iplot(figure)

## Accuracy vs. Sample Size of Weighted Freeflow Estimators By Level

In [ ]:
name = 'estimations_accuracy_level'

query = f"""
select sample_size, level_group, 
avg_accuracy as freeflow, roadtype_accuracy as roadtype_freeflow, best_accuracy as best_freeflow
from {config["athena_database"]}.{config["slug"]}_jams_ready_estimations_accuracy_level
order by sample_size
"""

save_path = query_and_save(query, name, config, conn)

In [ ]:
df = pd.read_csv(save_path)

In [ ]:
rolling = list(map(int, df.query('sample_size < 100000').groupby('level_group').count()['sample_size'].divide(10)))
groups = ['1', '2', '3', '4', '2-3-4', '3-4']

# Adds figures
figs = [df.query(f"level_group == '{group}'")\
        .query('sample_size < 100000')\
        .set_index('sample_size')\
        .drop(['level_group'], 1)\
        .rolling(rolling[i]).mean()\
        .figure()
     for i, group in enumerate(groups)]

# Add legend group to same traces 
for i in range(len(groups)):
    for j in range(len(figs[i]['data'])):
        figs[i]['data'][j]['legendgroup'] = str(j)
        figs[i]['data'][j]['showlegend'] = i == 0

# Instantiate figure as subplot
figure = cf.subplots(figs, shape=(len(groups), 1), 
                     subplot_titles=[ 'level_group ' + group + f' (Rolling Window {rolling[i]})' 
                                     for i, group in enumerate(groups)], 
                     vertical_spacing=0.05)

# Adjust height and add overall title
figure['layout']['height'] = len(groups) * 300
figure['layout']['title'] = 'Accuracy x Sample Size by Level (Weighted)'


# Edits layout of each subplot
for i in range(1, len(groups) + 1):
    
    if i == 1:
        i = ''
        

    figure['layout'][f'yaxis{i}']['title'] = 'Accuracy'
    figure['layout'][f'yaxis{i}']['range'] = [0.2, 1]

In [ ]:
plotly.offline.iplot(figure)

# Freeflow

## Distribution of Freeflow

In [ ]:
name = 'freeflow'

query = f"""
select freeflow
from {config["athena_database"]}.{config["slug"]}_jams_ready_segments_complete
limit 200000
"""

save_path = query_and_save(query, name, config, conn)

In [ ]:
name = 'population_size'

query = f"""
select count(freeflow)
from {config["athena_database"]}.{config["slug"]}_jams_ready_segments_complete
"""

population = pd.read_csv(query_and_save(query, name, config, conn)).values[0][0]

In [ ]:
query

In [ ]:
df = pd.read_csv(save_path)
sample_size_share = round(len(df) / population * 100, 0)
df.iplot(kind='histogram',
        title=f'Distribution of Freeflow for {round(len(df) / 1000, 0)}k Segments (~{sample_size_share}% ss)',
        xTitle='Freeflow Speed (km/h)',
        yTitle='Frequency'
        )

In [ ]:
df['qtt'] = 1
df.set_index('freeflow').sort_index().cumsum().divide(df.sum()['qtt']).iplot(
        title=f'Cumulative Distribution of Freeflow for {round(len(df) / 1000, 0)}k Segments (~{sample_size_share}% ss)',
        xTitle='Freeflow Speed (km/h)',
        yTitle='Cummulative %',
        width=5
        )

## Freeflow Estimation and Roadtype

In [ ]:
name = 'freeflow_by_roadtype_distribution'

query = f"""
select roadtype, freeflow freeflow
from {config["athena_database"]}.{config["slug"]}_jams_ready_segments_complete
limit 200000
"""

save_path = query_and_save(query, name, config, conn)

In [ ]:
df = pd.read_csv(save_path)
df['roadtype_name'] = df['roadtype'].apply(lambda x: roadtype_dict[x])

## Probability Distribution of Freeflow by Roadtype

In [ ]:
df.pivot(columns='roadtype_name', values='freeflow')\
    .iplot(kind='histogram', histnorm='probability',
          title=f'Probability Distribution of Freeflows By Roadtype for {round(len(df) / 1000, 0)}k Segments (~{sample_size_share}% ss)',
          yTitle='Probabilty', xTitle='Estimated Freeflow',
          )

## Cumulative distribution of Freeflow by Roadtype

In [ ]:
df.groupby(['freeflow', 'roadtype_name']).count().reset_index()\
.pivot(columns='roadtype_name', values='roadtype', index='freeflow')\
.cumsum().divide(df.groupby('roadtype_name').count()['roadtype'] / 100).iplot( 
    title='Cumulative distribution of Freeflow by Roadtype',
#     xTitle='Sample Size',
    yTitle='%',
    shape=(3,2),
    subplots=True,
#     shared_xaxes=True,
    subplot_titles=True,
    legend=False,
    mode='lines', 
    fill=True,
    width=0,
    xrange=[0,100]
    )

# Best Freeflow

## Distribution of  Best Freeflow

In [ ]:
name = 'best_freeflow'

query = f"""
select best_freeflow as freeflow
from {config["athena_database"]}.{config["slug"]}_jams_ready_segments_complete
limit 200000
"""

save_path = query_and_save(query, name, config, conn)

In [ ]:
name = 'population_size'

query = f"""
select count(best_freeflow)
from {config["athena_database"]}.{config["slug"]}_jams_ready_segments_complete
"""

population = pd.read_csv(query_and_save(query, name, config, conn)).values[0][0]

In [ ]:
query

In [ ]:
df = pd.read_csv(save_path)
sample_size_share = round(len(df) / population * 100, 0)
df.iplot(kind='histogram',
        title=f'Distribution of Freeflow for {round(len(df) / 1000, 0)}k Segments (~{sample_size_share}% ss)',
        xTitle='Freeflow Speed (km/h)',
        yTitle='Frequency'
        )

In [ ]:
df['qtt'] = 1
df.set_index('freeflow').sort_index().cumsum().divide(df.sum()['qtt']).iplot(
        title=f'Cumulative Distribution of Best Freeflow for {round(len(df) / 1000, 0)}k Segments (~{sample_size_share}% ss)',
        xTitle='Freeflow Speed (km/h)',
        yTitle='Cummulative %',
        width=5
        )

## Freeflow Estimation and Roadtype

In [ ]:
name = 'best_freeflow_by_roadtype_distribution'

query = f"""
select roadtype, best_freeflow freeflow
from {config["athena_database"]}.{config["slug"]}_jams_ready_segments_complete
limit 200000
"""

save_path = query_and_save(query, name, config, conn)

In [ ]:
df = pd.read_csv(save_path)
df['roadtype_name'] = df['roadtype'].apply(lambda x: roadtype_dict[x])

## Probability Distribution of Freeflow by Roadtype

In [ ]:
df.pivot(columns='roadtype_name', values='freeflow')\
    .iplot(kind='histogram', histnorm='probability',
          title=f'Probability Distribution of Best Freeflow By Roadtype for {round(len(df) / 1000, 0)}k Segments (~{sample_size_share}% ss)',
          yTitle='Probabilty', xTitle='Estimated Freeflow',
          )

## Cumulative distribution of Freeflow by Roadtype

In [ ]:
df.groupby(['freeflow', 'roadtype_name']).count().reset_index()\
.pivot(columns='roadtype_name', values='roadtype', index='freeflow')\
.cumsum().divide(df.groupby('roadtype_name').count()['roadtype'] / 100).iplot( 
    title='Cumulative distribution of Best Freeflow by Roadtype',
#     xTitle='Sample Size',
    yTitle='%',
    shape=(3,2),
    subplots=True,
#     shared_xaxes=True,
    subplot_titles=True,
    legend=False,
    mode='lines', 
    fill=True,
    width=0,
    xrange=[0,100]
    )

## Central Statistics By Roadtype

In [ ]:
name = 'freeflow_roadtype_average'

query = f"""
select roadtype, avg(freeflow) as avg, 
                 approx_percentile(freeflow, 0.5) as median
from {config["athena_database"]}.{config["slug"]}_jams_ready_segments_complete
group by roadtype
"""

save_path = query_and_save(query, name, config, conn)

In [ ]:
df = pd.read_csv(save_path)
df['roadtype_name'] = df['roadtype'].apply(lambda x: roadtype_dict[x])

In [ ]:
df.sort_values(by='roadtype')[['roadtype_name', 'avg', 'median']]